In [1]:
using SymPy

In [2]:
function _sym(x::Expr)
    y = x
    y.args .= _sym.(y.args)
    return  y
end

_sym(x::Number) = Sym(x)
_sym(x) = x

macro sym(x)
    return esc(_sym(x))
end

@sym (macro with 1 method)

In [3]:
function get_glrk_nodes(s)
    if s == 1
        c = @sym Vector([1//2])
    elseif s == 2
        c = @sym [1/2-√3/6, 1/2+√3/6]
    elseif s == 3
        c = @sym [1/2-√15/10,  1/2,        1/2+√15/10 ]
    elseif s == 4
        c = @sym [
          1/2-√(3/7+2*√30/35)/2,
          1/2-√(3/7-2*√30/35)/2,
          1/2+√(3/7-2*√30/35)/2,
          1/2+√(3/7+2*√30/35)/2
        ]
    else
        @error "GLRK nodes for " * string(s) * " stages not implemented."
    end

    return c
end

get_glrk_nodes (generic function with 1 method)

In [4]:
function get_lobatto_nodes(s)
    if s == 2
        c = @sym [0, 1]
    elseif s == 3
        c = @sym [0, 1//2, 1]
    elseif s == 4
        c = @sym [0,  (5-√5)/10,   (5+√5)/10,  1]
    elseif s == 5
        c = @sym [0,  1//2-√21/14,  1//2,  1//2+√21/14,  1]
    else
        @error "Lobatto nodes for " * string(s) * " stages not implemented."
    end

    return c
end

get_lobatto_nodes (generic function with 1 method)

In [15]:
function get_glrk_lobatto_tableau(s, σ=2)
    c = get_glrk_nodes(s)
    c̄ = get_lobatto_nodes(σ)
    a = [Sym("a_" * string(i) * string(j)) for i in 1:σ, j in 1:s]
    
    sol = Dict()

    for i in 1:σ
        eqs = []
        for k in 1:s
            eq = - c̄[i]^k / k
            for j in 1:s
                eq += a[i,j] * c[j]^(k-1)
            end
            push!(eqs, eq)
        end
        sol = merge(sol, solve(Sym.(eqs), a[i,:]))
    end
    
    A = [sol[a[i,j]] for i in axes(a,1), j in axes(a,2)]
end

get_glrk_lobatto_tableau (generic function with 2 methods)

In [16]:
get_glrk_lobatto_tableau(1)

2×1 Array{Sym,2}:
 0
 1

In [17]:
get_glrk_lobatto_tableau(2)

2×2 Array{Sym,2}:
   0    0
 1/2  1/2

In [18]:
get_glrk_lobatto_tableau(3)

2×3 Array{Sym,2}:
    0    0     0
 5/18  4/9  5/18

In [41]:
function get_glrk_lobatto_tableau_manual(s, σ=2)
    c = get_glrk_nodes(s)
    c̄ = get_lobatto_nodes(σ)
    a = [Sym("a_" * string(i) * string(j)) for i in 1:σ, j in 1:s]
    A = [Sym(0) for i in 1:σ, j in 1:s]
    
    sol = Dict()

    for i in 2:σ
        eqs = Sym[]
        for k in 1:s
            eq = - c̄[i]^k / k
            for j in 1:s
                eq += a[i,j] * c[j]^(k-1)
            end
            push!(eqs, eq)
        end

        for k in 1:s
            A[i,k] = solve(eqs[k], a[i,k])[1]
            for l in k+1:s
                eqs[l] = simplify(subs(eqs[l], a[i,k], A[i,k]))
            end
        end
        
        for k in s-1:-1:1
            for l in k+1:s
                A[i,k] = subs(A[i,k], a[i,l], A[i,l])
            end
        end
    end
    
    return simplify.(A)
end

get_glrk_lobatto_tableau_manual (generic function with 2 methods)

In [42]:
get_glrk_lobatto_tableau_manual(2)

2×2 Array{Sym,2}:
   0    0
 1/2  1/2

In [43]:
get_glrk_lobatto_tableau_manual(3)

2×3 Array{Sym,2}:
    0    0     0
 5/18  4/9  5/18

In [44]:
a = get_glrk_lobatto_tableau_manual(4)

2×4 Array{Sym,2}:
                  0  …                   0
 -sqrt(30)/72 + 1/4     -sqrt(30)/72 + 1/4

In [50]:
a[2,2] == a[2,3]

true

In [48]:
a[2,2] = a[2,3]

√30   1
─── + ─
 72   4

In [49]:
a

2×4 Array{Sym,2}:
                  0                  0                  0                   0
 -sqrt(30)/72 + 1/4  sqrt(30)/72 + 1/4  sqrt(30)/72 + 1/4  -sqrt(30)/72 + 1/4